In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
import requests
from datetime import datetime
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
from src.articles.ArticleAttributesService import ArticleAttributesService

In [4]:
aas = ArticleAttributesService()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from src.articles.ArticleService import ArticleService

In [6]:
article_id = '65dc6f3d1f6d9144de7ee6c4'

In [7]:
article = ArticleService.get_Article(article_id=article_id)

In [8]:
prompt = aas.generate_prompt(article_content=article.full_content)

In [9]:
prompt

'<|im_start|>system\nYou are an expert chief editor for a leading Indian business and financial content website. You evaluate critical attributes of articles to gatekeep content quality. The following are 19 attributes with the format of attribute (datatype): <instruction>\n        \'{"summary": "<Summarize the article, focusing on: a. Thoroughness: Include essential details and expand on headline points. b. Readability: Ensure proper grammar, 2-3 points, up to 80 words. c. Faithfulness: Don\'t mention any details which are not part of the article. d. Accuracy: Verify numbers, dates, and events. f. Structure: Single-line points preceded by relevant emoji and label. g. Format: Use !emoji! label: point format without preamble/postamble, adhering to a 3-point maximum. We will call this format as _summary_markdown_format_ -> \'!emoji1! label1: point1 \\n !emoji2! label2: point2. The platform is visited by users of all age groups and hence do not use any inappropriate content or emojis>", "

In [10]:
params = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.5,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.02,
    "return_full_text": False,
    "stop": ["###", "</s>", aas.tokenizer.eos_token]
}
request = {"inputs": prompt, "parameters": params, "stream": False}

In [11]:
from botocore.config import Config

In [12]:
import boto3

In [13]:
my_config = Config(
            region_name='ap-south-1',
            retries={
                'max_attempts': 5,
                'mode': 'standard'
            },
            max_pool_connections=40  # Increase the pool size
        )

        # Create a SageMaker Runtime client with the custom configuration
sess1 = boto3.session.Session()
sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)

In [14]:
response = sagemaker_runtime.invoke_endpoint(
            EndpointName=aas.llm_endpoint,
            Body=json.dumps(request),
            ContentType="application/json",
        )

In [15]:
k = response['Body'].read()

In [26]:
isinstance(json.loads(json.loads(k)[0]['generated_text']), dict)

False

In [59]:
l = json.loads(k)[0]['generated_text'].replace(prompt,'')

In [61]:
json.loads(json.loads(l))

{'summary': '!🚗! Renewal Process: The car insurance renewal process has become straightforward, with options for online and offline renewals. \n!💼! Update Details: Provide accurate details about your vehicle, driving record, and personal details during the renewal process. \n!💰! Renewal Benefits: Renewing car insurance provides financial security, legal compliance, and potential discounts.',
 'summary_critique': "The summary is accurate, thorough, and readable. It follows the _summary_markdown_format_ correctly, with relevant emojis and labels. The summary is faithful to the article's content and doesn't include any extraneous information.",
 'improved_summary': '!🚗! Renewal Process: Renewing car insurance online or offline has become hassle-free. \n!💼! Update Details: Ensure accurate details about your vehicle, driving record, and personal details during the renewal. \n!💰! Renewal Advantages: Renewing car insurance guarantees financial security, legal compliance, and potential discoun

In [13]:
aas.generate_prompt

In [3]:
r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/update_candidates')

In [27]:
r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json = {'articleId': '65dd503096d2a135a7054bdf'})

In [29]:
from sql.articles.MongoDBArticle import MongoDBArticle

In [71]:
from bson import ObjectId

In [91]:
def fetch_missing_summarized_articles(limit=None):
    query = {
        "is_premium_article": False,
        "content_type": 'ARTICLE',
        "cleaned_text": {"$exists": True, "$ne": None},
        "ai_generated_info.summary.OpenHermes_WatermelonUvulaMarigold": {"$exists": False},  # Check for the existence of the special_key,
        "ai_generated_info.summary": {"$exists": False},  # Check for the existence of the special_key,
        "published_time": {"$gt": "2024-01-20"},
          "source_id": {
                "$ne": ObjectId("6560ac8b9952a16bbc397223")
  },
    }
    collection = MongoDBArticle.get_collection()
    if isinstance(limit, int):
        documents = list(collection.find(query).limit(limit))
    else:
        documents = list(collection.find(query))
    for doc in documents:
        doc['article_id'] = str(doc['_id'])
        doc['source_id'] = str(doc['source_id'])
    return documents

In [92]:
missing_articles = fetch_missing_summarized_articles()

In [93]:
len(missing_articles)

109

In [94]:
missing_article_ids = [str(x['_id']) for x in missing_articles]

In [95]:
from concurrent.futures import ThreadPoolExecutor, as_completed

chunk_size = 20
num_chunks = len(missing_article_ids)//chunk_size + 1

In [96]:
def get_attributes(article_id):
    r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json = {'articleId': article_id})

In [97]:
import time
from datetime import datetime

In [98]:
from tqdm.notebook import tqdm

In [99]:
for i in range(num_chunks):
    article_chunk = missing_article_ids[i*chunk_size:(i+1)*chunk_size]
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_attributes, art_id) for art_id in article_chunk]
        for future in as_completed(futures):
            res = future.result()
    print(f'completed chunk {(i + 1) * chunk_size} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

completed chunk 20 articles at Feb 28, 2024 13:19 in 1.85 minutes
completed chunk 40 articles at Feb 28, 2024 13:21 in 1.6 minutes
completed chunk 60 articles at Feb 28, 2024 13:23 in 1.9 minutes
completed chunk 80 articles at Feb 28, 2024 13:25 in 1.7666666666666666 minutes
completed chunk 100 articles at Feb 28, 2024 13:27 in 1.9833333333333334 minutes
completed chunk 120 articles at Feb 28, 2024 13:27 in 0.8833333333333333 minutes


In [8]:
r.text

'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [5]:
def assign_article_to_cluster(article_id):
    r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/assign_cluster', json = {'articleId': article_id})
    if r.status_code == 200:
        return True, article_id
    else:
        return False, article_id

In [37]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [38]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT article_id FROM embeddings WHERE created_at > '2024-02-19'"
    cursor.execute(sql)
    results = cursor.fetchall()

In [39]:
embedded_articles = [x[0] for x in results]

In [40]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT article_id FROM article_to_cluster_mapping"
    cursor.execute(sql)
    results = cursor.fetchall()

In [41]:
clustered_articles = [x[0] for x in results]

In [42]:
pending_articles = list(set(embedded_articles) - set(clustered_articles))

In [43]:
len(pending_articles)

3

In [32]:
chunk_size = 32
num_chunks = len(pending_articles)//chunk_size + 1

In [33]:
completed_article_ids_new = []
pending_article_ids_new = []

In [34]:
for i in range(num_chunks):
    article_chunk = pending_articles[i*chunk_size:(i+1)*chunk_size]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = [executor.submit(assign_article_to_cluster, art_id) for art_id in article_chunk]
        for future in as_completed(futures):
            success, art_id = future.result()
            if success:  # Timeout or other errors
                completed_article_ids_new.append(art_id)
            else:
                pending_article_ids_new.append(art_id)
    print(f'completed chunk {(i + 1) * chunk_size} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Feb 27, 2024 06:02
completed chunk 32 articles at Feb 27, 2024 06:03 in 0.08333333333333333 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 64 articles at Feb 27, 2024 06:03 in 0.08333333333333333 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 96 articles at Feb 27, 2024 06:03 in 0.08333333333333333 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 128 articles at Feb 27, 2024 06:03 in 0.08333333333333333 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 160 articles at Feb 27, 2024 06:03 in 0.08333333333333333 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 192 articles at Feb 27, 2024 06:03 in 0.1 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 224 articles at Feb 27, 2024 06:03 in 0.08333333333333333 minutes
starting creation at Feb 27, 2024 06:03
completed chunk 256 articles at Feb 27, 2024 06:03 in 0.05 minutes
